# PyCity Schools Analysis

The purpose of this report is to analyze students scores on standardized tests for mathematics and reading to inform budgets and priorities for our next upcoming school year.

**Table of Contents:**<br>
* [District Summary](#district-summary)
* [School Summary](#school-summary)
* [Top Performing Schools (By Passing Rate)](#top-performing-schools-(by-passing-rate))
* [Bottom Performing Schools (By Passing Rate)](#bottom-performing-schools-(by-passing-rate))
* [Math Scores by Grade](#math-scores-by-grade)
* [Reading Scores by Grade](#reading-scores-by-grade)
* [Scores by School Spending](#scores-by-school-spending)
* [Scores by School Size](#scores-by-school-size)
* [Scores by School Type](#scores-by-school-type)

**Key Highlights:**

While we have schools who are very successful with the overwhelming majority of students passing both the math and reading portions of their standarding tests, we also have schools who are struggling to get their students to pass the standardized tests. We can have great impact on the students' performance by providing support for the math departments. 

<br>
<br>

*Note:*
*Code is available for inspection by clicking to the left of the respective code to open the cell. Code has been hidden to provide provide a report feel to this document.*
*Each of the required tables has been modified so that the columns display the data in a visually appealing manner. Original tables without the formatting are available in the "responses in template.ipynb" file.*

---

In [2]:
# Preview of Original Data

# Dependencies and Setup
import pandas as pd
from pathlib import Path
import locale

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete.head() # Commenting this out to avoid sharing information that identifies individuals, it is not necessary for the report to identify them


## District Summary

In [3]:
# Calculate the total number of unique schools
school_count = len(pd.unique(school_data_complete['school_name']))
# school_count

# Calculate the total number of students
student_count = len(school_data_complete.index) # This works because each row is a student
# student_count

# Calculate the total budget
# Create a DataFrame that only has school information
school_by_budget = school_data_complete[['school_name', 'budget']]
# Drop duplicates
school_by_budget = school_by_budget.drop_duplicates()
# Group by each unqiue school
school_group = school_by_budget.groupby(['school_name'])
# Add the budget together
total_budget = school_group['budget'].sum().sum()

# total_budget

# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
# average_math_score

# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
# average_reading_score

# Use the following to calculate the percentage of students who passed math (math scores greater than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
# passing_math_percentage

# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
# passing_reading_percentage

# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
# overall_passing_rate

# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame(
    {"Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_percentage],
    "% Passing Reading": [passing_reading_percentage],
    "% Overall Passing Rate": [overall_passing_rate]
    }
)

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary_visual = district_summary.copy()

# Adding Additional Formatting, but maintaining the full length of floats
district_summary_visual["Average Math Score"] = district_summary_visual["Average Math Score"].map("{:.2f}".format)
district_summary_visual["Average Reading Score"] = district_summary_visual["Average Reading Score"].map("{:.2f}".format)
district_summary_visual["% Passing Math"] = district_summary_visual["% Passing Math"].map("{:.2f}%".format)
district_summary_visual["% Passing Reading"] = district_summary_visual["% Passing Reading"].map("{:.2f}%".format)
district_summary_visual["% Overall Passing Rate"] = district_summary_visual["% Overall Passing Rate"].map("{:.2f}%".format)

In [4]:
# Report
print(f"PyCity is proud to have {school_count} schools in our school district and serve {student_count} students in our region.")
print(f"In total, our schools have a combined budget of ${total_budget:,}.")
print(f"The average scores of our students on standardized testing is {average_math_score:.2f} in math and {average_reading_score:.2f} in reading.")
print(f"The passing rate for math ({passing_math_percentage:.2f}%) is lower than the passing rate for reading ({passing_reading_percentage:.2f}%).")
print(f"Overall, {overall_passing_rate:.2f}% of our students passed both math and reading.")
print(f"See summary table below.")

PyCity is proud to have 15 schools in our school district and serve 39170 students in our region.
In total, our schools have a combined budget of $24,649,428.
The average scores of our students on standardized testing is 78.99 in math and 81.88 in reading.
The passing rate for math (74.98%) is lower than the passing rate for reading (85.81%).
Overall, 65.17% of our students passed both math and reading.
See summary table below.


In [5]:
district_summary_visual

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

Below is a brief summary of the performance of each school within our district. Note the size of the table reduces the visibility of the data.

In [19]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]

# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()
# per_school_counts

per_school_budget = school_data_complete[["school_name", "budget"]].groupby(["school_name"]).mean()["budget"]
# per_school_budget

# Calculate the total school budget and per capita spending
per_school_capita = per_school_budget / per_school_counts
# per_school_capita

# Calculate the average test scores
per_school_math = school_data_complete[["school_name", "math_score"]].groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete[["school_name", "reading_score"]].groupby(["school_name"]).mean()["reading_score"]
# per_school_math
# per_school_reading


# Calculate the number of schools with math scores of 70 or higher

# When I spoke with AskBCS, the tutor recommended this code below.
# The phrasing of the instructions for this item is confusing as it is asking about math scores for schools.
# However, schools do not have math scores. Students have math scores. 
# The instructions appears to be asking for a count of schools, a single number.
# It is unclear if the question is asking about the number of schools which a student scored higher than 70(?)
# or if it is looking at the average math score per school and if that average is greater than 70. 

school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70]
# school_passing_math

# Alternatively, if we were trying to get a count of the number of students at each school who passed with >= 70 in math, we would use the following code
per_school_passing_math = school_data_complete[school_data_complete["math_score"] >= 70].groupby(["school_name"]).count()["student_name"]
# per_school_passing_math

# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
# school_passing_reading

# And we could do the same for reading
per_school_passing_reading = school_data_complete[school_data_complete["reading_score"] >= 70].groupby(["school_name"]).count()["student_name"]
# per_school_passing_reading

# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

# Use the provided code to calculate the passing rates
per_school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
per_school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame(
    {"School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing Rate": overall_passing_rate
    }
)

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
# per_school_summary

per_school_summary_visual = per_school_summary.copy()

# Formatting
per_school_summary_visual["Average Math Score"] = per_school_summary_visual["Average Math Score"].map("{:.2f}".format)
per_school_summary_visual["Average Reading Score"] = per_school_summary_visual["Average Reading Score"].map("{:.2f}".format)
per_school_summary_visual["% Passing Math"] = per_school_summary_visual["% Passing Math"].map("{:.2f}%".format)
per_school_summary_visual["% Passing Reading"] = per_school_summary_visual["% Passing Reading"].map("{:.2f}%".format)
per_school_summary_visual["% Overall Passing Rate"] = per_school_summary_visual["% Overall Passing Rate"].map("{:.2f}%".format)

# Display the DataFrame
# per_school_summary_visual

In [20]:
per_school_summary_visual.columns

Index(['School Type', 'Total Students', 'Total School Budget',
       'Per Student Budget', 'Average Math Score', 'Average Reading Score',
       '% Passing Math', '% Passing Reading', '% Overall Passing Rate'],
      dtype='object')

In [21]:
per_school_summary_visual

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Highest-Performing Schools (by % Overall Passing)

Our top performing schools are below. At these schools, the majority of students pass both the math and reading exams. These schools are all Charter schools with fewer than 2300 students. 

In [8]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by=["% Overall Passing Rate"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

The schools below have the lowest passing rates in the district with approximately half of the students passing both the math and reading exams. These schools are all District schools with more than 2900 students.

In [9]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by=["% Overall Passing Rate"])
bottom_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Scores by Grade

Scores are typically consistent across grade levels at each school.

### Math Scores by Grade

In [10]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders[["school_name", "math_score"]].groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders[["school_name", "math_score"]].groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders[["school_name", "math_score"]].groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders[["school_name", "math_score"]].groupby(["school_name"]).mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame(
    {"9th": ninth_grade_math_scores,
    "10th": tenth_grader_math_scores,
    "11th": eleventh_grader_math_scores,
    "12th": twelfth_grader_math_scores
    }
)

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


### Reading Score by Grade 

In [11]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders[["school_name", "reading_score"]].groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders[["school_name", "reading_score"]].groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders[["school_name", "reading_score"]].groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders[["school_name", "reading_score"]].groupby(["school_name"]).mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame(
    {"9th": ninth_grade_reading_scores,
    "10th": tenth_grader_reading_scores,
    "11th": eleventh_grader_reading_scores,
    "12th": twelfth_grader_reading_scores
    }
)

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

Increasing spending per student does not relate to an increase in scores. To the contrary, there appears to be in inverse relationship between the amount of funding spent per student. A higher amount of spending per student is associated with lower passing rates. The relationship between amount spent per student and passing rates in math is particularly dramatic. Additional analyses that statistically control for the size of the school and/or class size would be valuable to further examine this relationship.  

In [12]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

# The template code above added formatting to the Per Student Budget column
# To proceed, the formatting must be removed from our new copy of the DataFrame
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace("$", "")
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].str.replace(",", "")
school_spending_df["Per Student Budget"] = school_spending_df["Per Student Budget"].astype(float)

# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df ["Per Student Budget"],
                                                            spending_bins,
                                                            labels=labels,
                                                            include_lowest=True)

# school_spending_df

#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df[["Spending Ranges (Per Student)", "Average Math Score"]].groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df[["Spending Ranges (Per Student)", "Average Reading Score"]].groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df[["Spending Ranges (Per Student)", "% Passing Math"]].groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df[["Spending Ranges (Per Student)", "% Passing Reading"]].groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df[["Spending Ranges (Per Student)", "% Overall Passing Rate"]].groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing Rate"]

# Assemble into DataFrame
spending_summary = pd.DataFrame(
    {"Average Math Score": spending_math_scores,
    "Average Reading Score": spending_reading_scores,
    "% Passing Math": spending_passing_math,
    "% Passing Reading": spending_passing_reading,
    "% Overall Passing Rate": overall_passing_spending
    }
)

# Display results
# spending_summary

# Make a copy to make visual changes to the columns
spending_summary_visual = spending_summary.copy()

# Formatting of each column
spending_summary_visual["Average Math Score"] = spending_summary_visual["Average Math Score"].map("{:.2f}".format)
spending_summary_visual["Average Reading Score"] = spending_summary_visual["Average Reading Score"].map("{:.2f}".format)
spending_summary_visual["% Passing Math"] = spending_summary_visual["% Passing Math"].map("{:.2f}%".format)
spending_summary_visual["% Passing Reading"] = spending_summary_visual["% Passing Reading"].map("{:.2f}%".format)
spending_summary_visual["% Overall Passing Rate"] = spending_summary_visual["% Overall Passing Rate"].map("{:.2f}%".format)

# spending_summary_visual

In [13]:
spending_summary_visual

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46%,96.61%,90.37%
$585-630,81.90,83.16,87.13%,92.72%,81.42%
$630-645,78.52,81.62,73.48%,84.39%,62.86%
$645-680,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

Larger schools appear to have lower performance in comparison to smaller schools. Additional information, such as the typical size of a class, would be valuable to examine the nuances of school size.

In [14]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], 
                                           size_bins,
                                           labels=labels,
                                           include_lowest=True)
# per_school_summary

# Calculate averages for the desired columns. 
size_math_scores = per_school_summary[["School Size", "Average Math Score"]].groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary[["School Size", "Average Reading Score"]].groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary[["School Size", "% Passing Math"]].groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary[["School Size", "% Passing Reading"]].groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary[["School Size", "% Overall Passing Rate"]].groupby(["School Size"]).mean()["% Overall Passing Rate"]

# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame(
    {"Average Math Score": size_math_scores,
    "Average Reading Score": size_reading_scores,
    "% Passing Math": size_passing_math,
    "% Passing Reading": size_passing_reading,
    "% Overall Passing Rate": size_overall_passing
    }
)

# Display results
# size_summary

size_summary_visual = size_summary.copy()

# Formatting of the columns
size_summary_visual["Average Math Score"] = size_summary_visual["Average Math Score"].map("{:.2f}".format)
size_summary_visual["Average Reading Score"] = size_summary_visual["Average Reading Score"].map("{:.2f}".format)
size_summary_visual["% Passing Math"] = size_summary_visual["% Passing Math"].map("{:.2f}%".format)
size_summary_visual["% Passing Reading"] = size_summary_visual["% Passing Reading"].map("{:.2f}%".format)
size_summary_visual["% Overall Passing Rate"] = size_summary_visual["% Overall Passing Rate"].map("{:.2f}%".format)

# size_summary_visual

In [15]:
size_summary_visual

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

Charter schools typically perform better than District schools. However, note that Charter schools are also typically smaller than District schools, so multiple factors may be at play. 

In [16]:

# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary[["School Type", "Average Math Score"]].groupby(["School Type"]).mean()
type_reading_scores = per_school_summary[["School Type", "Average Reading Score"]].groupby(["School Type"]).mean()
type_passing_math = per_school_summary[["School Type", "% Passing Math"]].groupby(["School Type"]).mean()
type_passing_reading = per_school_summary[["School Type", "% Passing Reading"]].groupby(["School Type"]).mean()
type_overall_passing = per_school_summary[["School Type", "% Overall Passing Rate"]].groupby(["School Type"]).mean()

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing Rate"]

# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame(
    {"Average Math Score": average_math_score_by_type,
    "Average Reading Score": average_reading_score_by_type,
    "% Passing Math": average_percent_passing_math_by_type,
    "% Passing Reading": average_percent_passing_reading_by_type,
    "% Overall Passing Rate": average_percent_overall_passing_by_type
    }
)

# Display results
# type_summary

type_summary_visual = type_summary.copy()

# Formatting of the columns
type_summary_visual["Average Math Score"] = type_summary_visual["Average Math Score"].map("{:.2f}".format)
type_summary_visual["Average Reading Score"] = type_summary_visual["Average Reading Score"].map("{:.2f}".format)
type_summary_visual["% Passing Math"] = type_summary_visual["% Passing Math"].map("{:.2f}%".format)
type_summary_visual["% Passing Reading"] = type_summary_visual["% Passing Reading"].map("{:.2f}%".format)
type_summary_visual["% Overall Passing Rate"] = type_summary_visual["% Overall Passing Rate"].map("{:.2f}%".format)

# type_summary_visual

In [17]:
type_summary_visual

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


**Report Complied By:**<br>S.E. Stegall-Rodriguez<br>Chief Data Scientist<br>PyCity School District

**Date:**<br>2023-04-24